In [3]:
import numpy as np
import pandas as pd 
import keras 
from keras.layers import *
from keras.models import *
from keras.activations import *
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import os
import itertools
import string
import tensorflow as tf
from itertools import product
import math
from utils import lex
from utils import yacc
from utils import cpp
import zipfile

# Here We Will Configure The Global Variables

In [ ]:
zip_name = "./data.zip"


cols = ["code", "block"]
# The first name is the name of the containing folder of the codes 
list_files = archive.namelist()[1:]
# Now We will make a scanner for the C++ language
scanner = lex.lex(cpp)
# If any thing was considered fault at the line i, we will consider all the lines [i - range_n, i + range_n) to be fault 
range_n = 4
lits = cpp.literals
toks = list(cpp.tokens)
toks.remove("CPP_WS")
toks.insert(0, "CPP_WS")
tok2n = dict(zip(toks + [i for i in lits], itertools.count()))
n2tok = dict(zip(itertools.count(), toks + [i for i in lits]))

max_v = 2147483647 - 1

WEIGHTS_FOR_LOSS = np.array([[2,0.5],[0.1,0.1]])

cons_per_line = 10


df1 = pd.read_csv(list_files[0], sep = "`")
df2 = pd.read_csv("examBefore.csv", sep = "`")
list1 = list(df1.columns)
list2 = list(df2.columns)
ind = -1
for i in range(len(list1)):
    if not (list1[i] == list2[i]):
        ind = i
        break
ind_2 = ind
ind_2 = -1